In [1]:
import requests
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import zipfile
from tqdm import tqdm
import base64
import pathlib

In [2]:
load_dotenv()

True

In [8]:
note = requests.get('https://centrala.ag3nts.org/dane/barbara.txt')

In [13]:
note.text.replace('\n', '')

'Podczas pobytu w Krakowie w 2019 roku, Barbara Zawadzka poznała swojego ówczesnego narzeczonego, a obecnie męża, Aleksandra Ragowskiego. Tam też poznali osobę prawdopodobnie powiązaną z ruchem oporu, której dane nie są nam znane. Istnieje podejrzenie, że już\xa0wtedy pracowali oni nad planami ograniczenia rozwoju sztucznej inteligencji, tłumacząc to względami bezpieczeństwa. Tajemniczy osobnik zajmował się\xa0także organizacją spotkań mających na celu podnoszenie wiedzy na temat wykorzystania sztucznej inteligencji przez programistów. Na spotkania te uczęszczała także Barbara.W okolicach 2021 roku Ragowski udał się do Warszawy celem spotkania z profesorem Andrzejem Majem. Prawdopodobnie nie zabrał ze sobą żony, a cel ich spotkania nie jest do końca jasny.Podczas pobytu w Warszawie, w instytucie profesora doszło do incydentu, w wyniku którego, jeden z laborantów - Rafał Bomba - zaginął. Niepotwierdzone źródła informacji podają jednak, że Rafał spędził około 2 lata, wynajmując pokój w p

In [15]:
client = OpenAI()

In [21]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f'Jesteś moim asystentem i musisz mi pomóc namierzyć Barbarę Zawadzką. \
                                        Oto notatka na jej temat: {note.text.replace('\n', '')}'},
        {"role": "user", "content": "Rozpisz mi wszystkie imiona (poza Barbarą oczywiście) oraz wszystkie miasta jakie pojawiają się w notatce \
            i przygotuj mi te dane w słowniku z kluczami names i cities i odpowiednio przypisanymi wartościami w listach. Zwróć sam czysty słownik, imiona mają być bez nazwisk"}
    ]
)
answer = response.choices[0].message.content

In [23]:
answer

'```python\n{\n    "names": ["Aleksander", "Andrzej", "Rafał"],\n    "cities": ["Kraków", "Warszawa"]\n}\n```'

In [45]:
hints = {
    'names': ["Aleksander", "Andrzej", "Rafal"],
    'cities': ["Krakow", "Warszawa"]
}

In [52]:
clues = {}

In [53]:
url_people = 'https://centrala.ag3nts.org/people'

for name in hints['names']:
    people_hint_response = requests.post(
        url=url_people,
        json={
            'apikey': os.getenv('AI_DEVS_3_API_KEY'),
            'query': name
        }
    )
    if people_hint_response.status_code == 200:
        clues[name] = people_hint_response.json()['message']

In [56]:
url_places = 'https://centrala.ag3nts.org/places'

for city in hints['cities']:
    city_hint_response = requests.post(
        url=url_places,
        json={
            'apikey': os.getenv('AI_DEVS_3_API_KEY'),
            'query': city
        }
    )
    if city_hint_response.status_code == 200:
        clues[city] = city_hint_response.json()['message']

In [57]:
clues

{'Aleksander': 'KRAKOW LUBLIN WARSZAWA',
 'Andrzej': 'WARSZAWA GRUDZIADZ',
 'Rafal': 'GRUDZIADZ WARSZAWA LUBLIN',
 'Krakow': 'ALEKSANDER BARBARA ADAM',
 'Warszawa': 'RAFAŁ ALEKSANDER ANDRZEJ'}

In [44]:
people_hint_response.json()['message']

'KRAKOW LUBLIN WARSZAWA'

In [95]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f'Jesteś moim asystentem i musisz mi pomóc namierzyć Barbarę Zawadzką. \
                                        Oto notatka na jej temat: {note.text.replace('\n', '')}'},
        {"role": "system", "content": f'Tu masz dodatkowe poszlaki: {clues}'},
        {"role": "user", "content": "Połącz wszystkie dane w jedną całość i uzupełnij brakujące informacje. \
            Kto był współpracownikiem Aleksandra i Barbary? Z kim widział się Rafał? Być może znalezienie informacji na ten temat pozwoli nam wytypować kolejne miejsce, \
            w którym warto poszukać Barbary. To nie Lublin ani Warszawa. Czy może to być Elbląg i dlaczego?"}
    ]
)
answer = response.choices[0].message.content

In [96]:
answer

'Biorąc pod uwagę dostępne informacje, spróbujmy połączyć wszystkie poszlaki:\n\n1. **Aleksander Ragowski:**\n   - Wcześniej przebywał w Krakowie, gdzie poznał Barbarę.\n   - W 2021 roku udał się do Warszawy na spotkanie z profesorem Andrzejem Majem.\n   - W poszlakach jest powiązany z Krakowem, Lublinem i Warszawą.\n\n2. **Barbara Zawadzka:**\n   - Była związana z Krakowem, gdzie mogła otrzymać notatki od Aleksandra po jego powrocie z Warszawy.\n   - Nie wiadomo, aby opuściła kraj, ale wiadomo, że nie przebywa w Lublinie ani Warszawie.\n\n3. **Profesor Andrzej Maj:**\n   - Przebywał w Warszawie, związany z Grudziądzem.\n   - Spotkał się z Aleksandrem w Warszawie.\n\n4. **Rafał Bomba:**\n   - Zaginął po incydencie w Warszawie i spędził dwa lata w nieznanym miejscu, prawdopodobnie wynajmując pokój w hotelu.\n   - Poszlaki wskazują na jego powiązania z Grudziądzem, Warszawą i Lublinem.\n   - Istnieje podejrzenie, że był powiązany z ruchem oporu.\n\n5. **Powiązania geograficzne:**\n   - K

In [92]:
solution_response = {
   "task": "loop",
   "apikey": os.getenv('AI_DEVS_3_API_KEY'),
   "answer": 'ELBLAG'
}

In [93]:
response = requests.post('https://centrala.ag3nts.org/report', json=solution_response)

In [94]:
response.json()

{'code': 0, 'message': '{{FLG:GOTCHA}}'}